<a href="https://colab.research.google.com/github/joaogabrielanuto/Laboratorio-de-Codigos/blob/main/Explorando_Drawdowns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

---
# **Aprofundando em uma das métricas de risco mais importantes: Drawdowns**
---

In [ ]:
!pip install yfinance
!pip install vectorbt

In [ ]:
import pandas as pd
import numpy as np

import yfinance as yf
import vectorbt as vbt

In [ ]:
dados_ativo = vbt.YFData.download('PETR4.SA', start='2000-01-01').get(['Close'])
dados_ativo['Data'] = pd.to_datetime(dados_ativo.index.date)
dados_ativo.head()

<ipython-input-4-d24676a8cf1c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_ativo['Data'] = pd.to_datetime(dados_ativo.index.date)


,Close,Data
Date,,
2000-01-03 02:00:00+00:00,1.872028,2000-01-03
2000-01-04 02:00:00+00:00,1.768469,2000-01-04
2000-01-05 02:00:00+00:00,1.750624,2000-01-05
2000-01-06 02:00:00+00:00,1.744570,2000-01-06
2000-01-07 02:00:00+00:00,1.752537,2000-01-07


In [ ]:
df_dias_analise = pd.date_range(start='2000-01-01', end=pd.Timestamp.today(), freq='D').to_frame(name='Data')
preco_ativo = pd.merge(df_dias_analise, dados_ativo, on='Data', how='outer')
preco_ativo = preco_ativo.fillna(method='ffill')
preco_ativo = preco_ativo.set_index('Data', drop=True)
preco_ativo = preco_ativo.dropna()
preco_ativo

,Close
Data,
2000-01-03,1.872028
2000-01-04,1.768469
2000-01-05,1.750624
2000-01-06,1.744570
2000-01-07,1.752537
...,...
2023-01-08,23.740000
2023-01-09,23.870001
2023-01-10,24.090000


In [ ]:
analise_ativo = vbt.Portfolio.from_holding(preco_ativo.Close, freq='D', init_cash=preco_ativo.Close.iloc[0])

In [ ]:
analise_ativo.drawdowns.plot(top_n=3).update_layout(yaxis_title='<b>Patrimônio inicial (%)', width=1000, height=600, font=dict(size=20), margin=dict(l=20, r=20, t=20, b=20)) \
                                     .update_traces(marker=dict(size=20)).show()

In [ ]:
analise_ativo.drawdowns.plot(top_n=100).update_layout(yaxis_title='<b>Patrimônio inicial (%)', width=1000, height=600, font=dict(size=20), margin=dict(l=20, r=20, t=20, b=20)) \
                                     .update_traces(marker=dict(size=12)).show()

In [ ]:
analise_ativo.drawdowns.records_readable

,Drawdown Id,Column,Peak Timestamp,Start Timestamp,Valley Timestamp,End Timestamp,Peak Value,Valley Value,End Value,Status
0,0,Close,2000-01-03,2000-01-04,2000-01-31,2000-03-01,100.000000,86.825523,100.000000,Recovered
1,1,Close,2000-03-01,2000-03-02,2000-03-02,2000-03-03,100.000000,99.370251,102.774454,Recovered
2,2,Close,2000-03-09,2000-03-10,2000-04-24,2000-06-21,108.919150,85.106362,109.582993,Recovered
3,3,Close,2000-06-28,2000-06-29,2000-06-29,2000-07-03,116.808486,113.191479,119.148952,Recovered
4,4,Close,2000-07-04,2000-07-05,2000-08-09,2000-09-11,121.702133,95.851075,122.553219,Recovered
...,...,...,...,...,...,...,...,...,...,...
71,71,Close,2022-08-09,2022-08-10,2022-08-11,2022-08-12,1416.908430,1379.611448,1482.061055,Recovered
72,72,Close,2022-08-18,2022-08-19,2022-08-19,2022-08-23,1561.982984,1482.995866,1562.917796,Recovered
73,73,Close,2022-08-24,2022-08-25,2022-08-25,2022-08-26,1572.265399,1555.439713,1572.265399,Recovered
74,74,Close,2022-08-29,2022-08-30,2022-09-28,2022-10-19,1611.525331,1368.020502,1655.458912,Recovered


**Pior cenário**

In [ ]:
analise_ativo.drawdowns.max_drawdown()

-0.9014476176711628

In [ ]:
analise_ativo.drawdowns.max_duration()

Timedelta('4995 days 00:00:00')

In [ ]:
analise_ativo.drawdowns.decline_duration.max()

2805.0

In [ ]:
analise_ativo.drawdowns.recovery_duration.max()

2191.0

**EDA dos parâmetros**

In [ ]:
analise_ativo.drawdowns.drawdown.bottom_n(10).describe()

count    10.000000
mean     -0.330226
std       0.213263
min      -0.901448
25%      -0.323391
50%      -0.241259
75%      -0.226220
max      -0.212413
Name: Close, dtype: float64

In [ ]:
analise_ativo.drawdowns.decline_duration.top_n(10).describe()

count      10.000000
mean      373.600000
std       865.772256
min        39.000000
25%        47.250000
50%        52.000000
75%        80.000000
max      2805.000000
Name: Close, dtype: float64

**Drawdown atual (active)**

In [ ]:
analise_ativo.drawdowns.active.avg_duration()

Timedelta('81 days 00:00:00')

In [ ]:
analise_ativo.drawdowns.active.peak_val.values

array([33.00302887])

In [ ]:
analise_ativo.drawdowns.active.avg_drawdown()

-0.34945366989899845

In [ ]:
analise_ativo.drawdowns.active_recovery_duration()

Timedelta('29 days 00:00:00')

In [ ]:
analise_ativo.drawdowns.active_drawdown()

array(-0.26430993)